# 데이터 로드

In [ ]:
import pandas as pd

# # # 긍정 부정 사전 # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
txt = '/content/drive/MyDrive/t-NPS_진일님/키워드.txt'            

knu = pd.read_csv(txt, delimiter = '\t')            

knu_neg = knu['단어'][knu['점수']<= -1].tolist()
knu_pos = knu['단어'][knu['점수']>= 0].tolist()
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 



# # # # # 원본 텍스트 데이터 가져오기 # # # # # # # # # # # # # # # # # # # # # 
txt = '/content/drive/MyDrive/t-NPS_진일님/긍정부정.txt'

text = pd.read_csv(txt, delimiter = '\t')
# # # # # # # # # # # # # # # # # # # # ## # # # # # # # # # # # # # # # # # # 

In [ ]:
# # # # # # # # # 네이버 맞춤범 검사기 설치 런타임 종료되면 다시 재설치 (띄어쓰기까지 지원)
# !pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
from hanspell import spell_checker

new_text = []
for i in text['문항10항목']:
    spelled_sent = spell_checker.check(i)
    new_text.append(spelled_sent.checked)

text['맞춤법'] = new_text
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # 라벨 붙이기
full = []
score = []
for txt in text['맞춤법']:
    for idx, i in enumerate(knu['단어']):
        if i in txt:
            score.append(knu['점수'][idx])
    full.append(sum(score))
    # print(sum(score), '초기화됩니다.')
    score = []
# print(len(full), len(text))
text['점수'] = full
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
full = []
neg = []
stack = []
for txt in text['맞춤법']:
    for idx, i in enumerate(knu_neg):
        if i in txt:
            neg.append(i)
    full.append(neg)
    # print(len(neg))
    if len(neg) >= 1:
        stack.append('불만')
    else:
        stack.append(0)
    # print(sum(score), '초기화됩니다.')
    neg = []
text['불만'] = stack
text['불만단어'] = full
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
full = []
pos = []
stack = []
for txt in text['맞춤법']:
    for idx, i in enumerate(knu_pos):
        if i in txt:
            pos.append(i)
    full.append(pos)
    # print(full)
    # print(len(neg))
    if len(pos) >= 1:
        stack.append('칭찬')
    else:
        stack.append(0)
    # print(sum(score), '초기화됩니다.')
    pos = []
text['칭찬'] = stack
text['칭찬단어'] = full

* 부정 --> 음수 : TP
* 긍정 --> 양수 : TN
* 긍정 --> 음수 : FP
* 부정 --> 양수 : FN

In [ ]:
threshlod_num = 0
text['분류'] = 'none'
text['분류'].loc[(text['구분']=='부정')&(text['점수']<= threshlod_num)] = 'TP'
text['분류'].loc[(text['구분']=='부정')&(text['점수']> threshlod_num)] = 'FN'
text['분류'].loc[(text['구분']=='긍정')&(text['점수']<= threshlod_num)] = 'FP'
text['분류'].loc[(text['구분']=='긍정')&(text['점수']> threshlod_num)] = 'TN'

text['설명'] = 'none'
text['설명'].loc[text['분류']=='TP'] = '부정 --> 음수'
text['설명'].loc[text['분류']=='FN'] = '부정 --> 양수'
text['설명'].loc[text['분류']=='FP'] = '긍정 --> 음수'
text['설명'].loc[text['분류']=='TN'] = '긍정 --> 양수'

In [ ]:
# # # # # # # # # # # # # # # # # # # # 정확도 # # # # # # # # # # # # # # # # # # # # # # # # 
print('정확도:',(len(text['분류'][text['분류']=='TN'])+len(text['분류'][text['분류']=='TP']))/len(text))
print('\n')




# # # 부정을 부정으로 예측 # # # # 
print('부정을 부정으로 예측:',len(text['분류'][text['분류']=='TP']) / (len(text['분류'][text['분류']=='TP']) + len(text['분류'][text['분류']=='FN'])))

print('\n')
# # # 긍정을 긍정으로 예측 # # # # 
print('긍정을 긍정으로 예측:', len(text['분류'][text['분류']=='TN']) / (len(text['분류'][text['분류']=='TN']) + len(text['분류'][text['분류']=='FP'])))

In [ ]:
text.to_csv('/content/drive/MyDrive/t-NPS_진일님/현업사전기반감성분석_최종결과.csv', encoding = 'utf-8-sig')

In [ ]:
text[['맞춤법', '불만단어', '칭찬단어', '점수', '설명','구분']][text['분류']=='FN'].to_csv('/content/drive/MyDrive/t-NPS_진일님/부정을긍정으로판단.csv', encoding = 'utf-8-sig')
text[['맞춤법', '불만단어', '칭찬단어', '점수', '설명','구분']][text['분류']=='FP'].to_csv('/content/drive/MyDrive/t-NPS_진일님/긍정을부정으로판단.csv', encoding = 'utf-8-sig')
text[['맞춤법', '불만단어', '칭찬단어', '점수', '설명','구분']][text['분류']=='TP'].to_csv('/content/drive/MyDrive/t-NPS_진일님/부정을부정으로판단.csv', encoding = 'utf-8-sig')
text[['맞춤법', '불만단어', '칭찬단어', '점수', '설명','구분']][text['분류']=='TN'].to_csv('/content/drive/MyDrive/t-NPS_진일님/긍정을긍정으로판단.csv', encoding = 'utf-8-sig')

In [ ]:
# # # # #  # # # #  그래프 확인 # # # #  # # # #  # # # # 

In [ ]:
# Y axis as density
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns


plt.subplot(2, 1, 1) 
plt.hist(text['점수'][text['구분']== '부정'], bins=20)
plt.title('bad')

plt.subplot(2, 1, 2) 
plt.hist(text['점수'][text['구분']== '긍정'], bins=20)
plt.title('good')

# n, bins, patches = plt.hist(text['score'][text['구분']== '지속불편'], bins=10, density=True)

In [ ]:
import numpy as np

print('부정 평균 :', round(np.mean(text['점수'][text['구분']== '부정'].tolist()), 3), '\n'
      '중앙값 :', np.median(text['점수'][text['구분']== '부정'].tolist()),'\n'
     '최대값 :', max(text['점수'][text['구분']== '부정']),'\n'
    '최소값 :', min(text['점수'][text['구분']== '부정']))

print('\n','\n')

print('긍정 평균 :', round(np.mean(text['점수'][text['구분']== '긍정'].tolist()), 3), '\n'
      '중앙값 :', np.median(text['점수'][text['구분']== '긍정'].tolist()),'\n'
     '최대값 :', max(text['점수'][text['구분']== '긍정']),'\n'
    '최소값 :', min(text['점수'][text['구분']== '긍정']))